<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">LLM Ops - Tracing, Evaluation, and Analysis</h1>

In this tutorial we will learn how to build, observe, and evaluate a LLM powered application. 

It has the following sections:

1. Understanding LLM powered applications
2. Building a LLM powered application
3. Observing the application using traces
4. Evaluating the application using LLM Evals
5. Exploring and and troubleshooting the application using UMAP projection and clustering


## Understanding LLM powered applications

Building software with LLMs, or any machine learning model, is [fundamentally different](https://karpathy.medium.com/software-2-0-a64152b37c35). Rather than compiling source code into binary to run a series of commands, we need to navigate datasets, embeddings, prompts, and parameter weights to generate consistent accurate results. LLM outputs are probabilistic and therefor don't produce the same predictable outcome every time.

